# Project 2

In [2]:
from __future__ import print_function
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import sys
import networkx as nx
import pandas
from math import lgamma

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
smallDf = pandas.read_csv('small2.csv')

In [4]:
list(smallDf)

['s', 'a', 'r', 'sp']

In [5]:
smallDf

,s,a,r,sp
0,12,3,0,22
1,22,4,0,12
2,12,2,0,11
3,11,3,0,21
4,21,1,0,21
5,21,4,0,11
6,11,3,0,1
7,1,3,0,11
8,11,4,0,12
9,12,3,0,11


In [16]:
states = smallDf.s.unique()
actions = [1,2,3,4]

In [ ]:
g=nx.Graph()
for s in states:
    g.add_node(s, counts={1:0,2:0,3:0,4:0}, actions={})

In [74]:
T = {}
T_prob = {}
R = {}
U = {}

for s in states:
    T[s] = {}
    T_prob[s] = {}
    R[s] = 0
    U[s] = 0

In [63]:
df = smallDf.groupby(['s', 'a', 'sp']).size().reset_index(name='counts')

In [75]:
for row in df.itertuples():
    if row.a not in T[row.s]:
        T[row.s][row.a] = {}
        T_prob[row.s][row.a] = {}
    T[row.s][row.a][row.sp] = row.counts

In [76]:
for s in states:
    for a in actions:
        sum_val = sum(T[s][a].values())
        poss_states = T[s][a].keys()
        for sp in poss_states:
            T_prob[s][a][sp] = T[s][a][sp]/sum_val

In [66]:
rdf = smallDf[smallDf['r'] > 0].groupby(['r', 'sp']).size().reset_index(name='counts')

In [67]:
rdf

,r,sp,counts
0,3,13,127
1,3,22,119
2,3,24,123
3,3,33,126
4,10,68,104
5,10,77,139
6,10,79,111
7,10,88,123


In [77]:
for row in rdf.itertuples():
    R[row.sp] = row.r

In [96]:
'''function valueIteration(mdp::MDP, numIterations::Integer; discount = 0.9)
    (S, A, T, R) = locals(mdp)
    U = zeros(Float64, length(S))
    for t = 1:numIterations
        Q(s0, a) = R(s0, a) + discount*sum(s1 ->T(s0, a, s1)*U[s1], S)
        U = [maximum(a->Q(s0,a), A) for s0 in S]
    end
    U
end
function extractPolicy(mdp::MDP, U::Vector)
    (S, A, T, R, discount) = locals(mdp)    
    Q(s0, a) = R(s0, a) + discount*sum(s1->T(s0, a, s1)*U[s1], S)
    [A[indmax([Q(s0,a) for a in A])] for s0 in S]
end;
'''
discount = 0.95

    
def policy_iter(iters=10):
    policy = {}
    U = {}
    for s in states:
        U[s] = 0
    for i in range(iters):
        for s in states:
            action_results = []
            for a in actions:
                sp_util = []
                for sp in T_prob[s][a].keys():
                    #print(s,a,sp)
                    #print("u", U[sp])
                    #print("t", T_prob[s][a][sp])
                    sp_util.append(T_prob[s][a][sp]*U[sp])
                action_results.append(sum(sp_util))
            U[s] = R[s] + discount*max(action_results)
            policy[s] = np.argmax(action_results)+1
    return policy, U

In [115]:
policy, utility = policy_iter(5)

In [110]:
policy

{1: 2,
 2: 2,
 3: 3,
 4: 3,
 5: 3,
 6: 3,
 7: 3,
 8: 3,
 9: 3,
 10: 3,
 11: 2,
 12: 2,
 13: 3,
 14: 3,
 15: 3,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 3,
 21: 2,
 22: 2,
 23: 2,
 24: 3,
 25: 3,
 26: 3,
 27: 3,
 28: 3,
 29: 3,
 30: 3,
 31: 2,
 32: 2,
 33: 2,
 34: 2,
 35: 3,
 36: 3,
 37: 3,
 38: 3,
 39: 3,
 40: 3,
 41: 2,
 42: 2,
 43: 2,
 44: 2,
 45: 3,
 46: 3,
 47: 3,
 48: 3,
 49: 3,
 50: 3,
 51: 2,
 52: 2,
 53: 3,
 54: 2,
 55: 2,
 56: 3,
 57: 3,
 58: 3,
 59: 3,
 60: 1,
 61: 2,
 62: 2,
 63: 2,
 64: 2,
 65: 2,
 66: 2,
 67: 3,
 68: 3,
 69: 3,
 70: 1,
 71: 2,
 72: 2,
 73: 2,
 74: 2,
 75: 2,
 76: 2,
 77: 2,
 78: 3,
 79: 1,
 80: 1,
 81: 2,
 82: 2,
 83: 2,
 84: 2,
 85: 2,
 86: 2,
 87: 2,
 88: 4,
 89: 4,
 90: 1,
 91: 2,
 92: 2,
 93: 2,
 94: 2,
 95: 2,
 96: 2,
 97: 2,
 98: 4,
 99: 4,
 100: 4}

In [116]:
def save_policy(policy, filename):
    policy_sorted = []
    for s in states:
        policy_sorted.append(str(policy[s]))
    policy_string = '\n'.join(policy_sorted)
    with open(filename, 'w') as f:
        f.write(policy_string)

In [117]:
save_policy(policy, 'small.policy')

In [113]:
mediumDf = pandas.read_csv('medium2.csv')

In [118]:
mediumDf

,s,a,r,sp
0,24715,1,-225,24214
1,24214,1,-225,23713
2,23713,1,-225,23212
3,23212,1,-225,22711
4,22711,1,-225,22709
5,22709,1,-225,22207
6,22207,1,-225,21705
7,21705,1,-225,21703
8,21703,1,-225,21200
9,21200,1,-225,21197
